<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Глубокое обучение. Часть 2
# Домашнее задание по теме "Механизм внимания"

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

В этом задании вы будете решать задачу классификации математических задач по темам (многоклассовая классификация) с помощью Transformer.

В качестве датасета возьмем датасет математических задач по разным темам. Нам необходим следующий файл:

[Файл с классами](https://docs.google.com/spreadsheets/d/13YIbphbWc62sfa-bCh8MLQWKizaXbQK9/edit?usp=drive_link&ouid=104379615679964018037&rtpof=true&sd=true)

**Hint:** не перезаписывайте модели, которые вы получите на каждом из этапов этого дз. Они ещё понадобятся.

### Задание 1 (2 балла)

Напишите кастомный класс для модели трансформера для задачи классификации, использующей в качествке backbone какую-то из моделей huggingface.

Т.е. конструктор класса должен принимать на вход название модели и подгружать её из huggingface, а затем использовать в качестве backbone (достаточно возможности использовать в качестве backbone те модели, которые упомянуты в последующих пунктах)

In [1]:
!pip install transformers datasets evaluate accelerate >/dev/null

In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch import nn
import torch
from typing import Union
from transformers import DataCollatorWithPadding
import evaluate
import pandas as pd

2024-04-21 01:12:25.326016: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 01:12:25.326154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 01:12:25.451231: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
path_to_data = "/kaggle/input/math-multiclass-classification"
data_problems = pd.read_csv(f"{path_to_data}/data_problems.csv").drop('Unnamed: 0',axis=1)
data_problems_translated = pd.read_csv(f"{path_to_data}/data_problems_translated.csv").drop('Unnamed: 0',axis=1)

In [4]:
data_problems

,Задача,Тема
0,Между девятью планетами Солнечной системы введ...,Графы
1,"В стране Цифра есть 9 городов с названиями 1, ...",Графы
2,"В государстве 100 городов, и из каждого из них...",Графы
3,"В классе 30 человек. Может ли быть так, что 9 ...",Графы
4,В городе Маленьком 15 телефонов. Можно ли их с...,Графы
...,...,...
5268,Дьявол предлагает Человеку сыграть в следующую...,Дирихле
5269,По шоссе в одном направлении едут 10 автомобил...,Дирихле
5270,В пространстве имеются 30 ненулевых векторов. ...,Дирихле
5271,На плоскости даны две точки A и B. Пусть C – н...,Дирихле


In [5]:
data_problems_translated

,problem_text,topic
0,To prove that the sum of the numbers of the ex...,number_theory
1,( b) Will the statement of the previous challe...,number_theory
2,The quadratic three-member graph with the coef...,polynoms
3,Can you draw on the surface of Rubik's cube a ...,combinatorics
4,"Dima, who came from Vrunlandia, said that ther...",graphs
...,...,...
5268,Can the number n! end with the figures of 76.0...,number_theory
5269,Find all natural m and n for which m! + 12 = n2.,number_theory
5270,"The correct 5,000-column has a 2001 vertebrate...",dirichlet
5271,Find the maximum number N for which there are ...,number_theory


In [6]:
data_problems =data_problems.rename(columns={ "Задача": "text", "Тема": "label"})

In [7]:
data_problems

,text,label
0,Между девятью планетами Солнечной системы введ...,Графы
1,"В стране Цифра есть 9 городов с названиями 1, ...",Графы
2,"В государстве 100 городов, и из каждого из них...",Графы
3,"В классе 30 человек. Может ли быть так, что 9 ...",Графы
4,В городе Маленьком 15 телефонов. Можно ли их с...,Графы
...,...,...
5268,Дьявол предлагает Человеку сыграть в следующую...,Дирихле
5269,По шоссе в одном направлении едут 10 автомобил...,Дирихле
5270,В пространстве имеются 30 ненулевых векторов. ...,Дирихле
5271,На плоскости даны две точки A и B. Пусть C – н...,Дирихле


In [8]:
data_problems.label.value_counts()

label
Теория чисел     2396
Комбинаторика    1020
Дирихле           441
Многочлен         426
Графы             384
Геометрия         371
Инвариант         235
Name: count, dtype: int64

In [9]:
classes = data_problems.label.unique()
num_labels = len(classes)
id2label = {i:name for i,name in enumerate(classes)}
label2id = {name:i for i,name in id2label.items()}
classes,num_labels,id2label,label2id

(array(['Графы', 'Геометрия', 'Многочлен', 'Комбинаторика', 'Теория чисел',
        'Инвариант', 'Дирихле'], dtype=object),
 7,
 {0: 'Графы',
  1: 'Геометрия',
  2: 'Многочлен',
  3: 'Комбинаторика',
  4: 'Теория чисел',
  5: 'Инвариант',
  6: 'Дирихле'},
 {'Графы': 0,
  'Геометрия': 1,
  'Многочлен': 2,
  'Комбинаторика': 3,
  'Теория чисел': 4,
  'Инвариант': 5,
  'Дирихле': 6})

In [10]:
from datasets import Dataset,DatasetDict

In [11]:
def create_ds(df):
    ds = Dataset.from_pandas(df)
    ds = ds.class_encode_column("label")
    train_test = ds.train_test_split(test_size=0.2,stratify_by_column='label')
    ds = DatasetDict({
        'train': train_test['train'],
        'test': train_test['test'],
    })
    return ds


In [12]:
ds =create_ds(data_problems)
ds

Casting to class labels:   0%|          | 0/5273 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4218
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1055
    })
})

In [13]:
def wrapper_preprocess_function(tokenizer):
    def preprocess_function(examples):

        return tokenizer(examples["text"], truncation=True,max_length=512)
    return preprocess_function

In [14]:

def to_trans(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return TransformerClassificationModel(result)
    return wrapper

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device
# не понимаю зачем нам обертка если есть очень классный api от transformers, 

class TransformerClassificationModel:
    @staticmethod
    def make(base_transformer_model: Union[str, nn.Module], num_labels:int,id2label:dict,label2id:dict):
        if isinstance(base_transformer_model, str):
            return AutoModelForSequenceClassification.from_pretrained(base_transformer_model, num_labels=num_labels, id2label=id2label, label2id=label2id)
        else:
            raise Exception()

    

### Задание 2 (1 балл)

Напишите функцию заморозки backbone у модели (если необходимо, возвращайте из функции модель)

In [15]:
def freeze_backbone_function(model: TransformerClassificationModel, params_names_starts:list[str]):
    for name, param in model.named_parameters():
        for name_laeyr in params_names_starts:
            if name.startswith(name_laeyr): 
                param.requires_grad = False



In [16]:
def show_freeze(model: TransformerClassificationModel):
    for name, param in model.named_parameters():
        print(name, param.requires_grad)


### Задание 3 (2 балла)

Напишите функцию, которая будет использована для тренировки (дообучения) трансформера (TransformerClassificationModel). Функция должна поддерживать обучение с замороженным и размороженным backbone.

In [17]:
from sklearn.metrics import f1_score
import evaluate
f1 = evaluate.load('f1')
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    ff = f1.compute(predictions=preds, references=labels, average="macro")

    
    return ff


In [18]:
import copy

def train_transformer(transformer_model,ds,tokenizer,compute_metrics,training_args, params_names_starts:list[str]):
    model = copy.copy(transformer_model)
    freeze_backbone_function(model,params_names_starts)
    preprocess_function = wrapper_preprocess_function(tokenizer)
    tokenized_imdb = ds.map(preprocess_function, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_imdb["train"],
        eval_dataset=tokenized_imdb["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    return model

### Задание 4 (1 балл)

Проверьте вашу функцию из предыдущего пункта, дообучив двумя способами
*cointegrated/rubert-tiny2* из huggingface.

In [19]:
rubert_tiny_freeze = TransformerClassificationModel.make("cointegrated/rubert-tiny2",num_labels,id2label,label2id)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
show_freeze(rubert_tiny_freeze)

bert.embeddings.word_embeddings.weight True
bert.embeddings.position_embeddings.weight True
bert.embeddings.token_type_embeddings.weight True
bert.embeddings.LayerNorm.weight True
bert.embeddings.LayerNorm.bias True
bert.encoder.layer.0.attention.self.query.weight True
bert.encoder.layer.0.attention.self.query.bias True
bert.encoder.layer.0.attention.self.key.weight True
bert.encoder.layer.0.attention.self.key.bias True
bert.encoder.layer.0.attention.self.value.weight True
bert.encoder.layer.0.attention.self.value.bias True
bert.encoder.layer.0.attention.output.dense.weight True
bert.encoder.layer.0.attention.output.dense.bias True
bert.encoder.layer.0.attention.output.LayerNorm.weight True
bert.encoder.layer.0.attention.output.LayerNorm.bias True
bert.encoder.layer.0.intermediate.dense.weight True
bert.encoder.layer.0.intermediate.dense.bias True
bert.encoder.layer.0.output.dense.weight True
bert.encoder.layer.0.output.dense.bias True
bert.encoder.layer.0.output.LayerNorm.weight True


In [21]:
import os
os.environ["WANDB_DISABLED"] = "true"
import wandb
wandb.init(mode="disabled")
training_args_rubert_tiny_freeze = TrainingArguments(
    output_dir="rubert_tiny_freeze",
    learning_rate=2e-5,
    per_device_train_batch_size=100,
    per_device_eval_batch_size=100,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none",
)
training_args_rubert_tiny_freeze.device

device(type='cuda', index=0)

In [22]:

tokenizer_rubert_tiny = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
rubert_tiny_finetuned_with_freezed_backbone = train_transformer(rubert_tiny_freeze,ds,tokenizer_rubert_tiny,compute_metrics,training_args_rubert_tiny_freeze, ['model.bert'])



tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/4218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,No log,1.601077,0.089344
2,No log,1.492850,0.089344
3,No log,1.412803,0.236223
4,No log,1.366441,0.258673
5,No log,1.351586,0.258505


In [23]:
import gc
rubert_tiny_finetuned_with_freezed_backbone=None
rubert_tiny_freeze = None
training_args_rubert_tiny_freeze = None 
torch.cuda.empty_cache()
gc.collect()

63

In [24]:
training_args_rubert_tiny = TrainingArguments(
    output_dir="rubert_tiny",
    learning_rate=2e-5,
    per_device_train_batch_size=100,
    per_device_eval_batch_size=100,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none",
)
rubert_tiny = TransformerClassificationModel.make("cointegrated/rubert-tiny2",num_labels,id2label,label2id)
rubert_tiny_full_finetuned = train_transformer(rubert_tiny,ds,tokenizer_rubert_tiny,compute_metrics,training_args_rubert_tiny,[])



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,No log,1.605305,0.089344
2,No log,1.488844,0.138380
3,No log,1.410221,0.222993
4,No log,1.365152,0.236344
5,No log,1.350593,0.241224


In [25]:
training_args_rubert_tiny = None
rubert_tiny = None 
training_args_rubert_tiny = None
tokenizer_rubert_tiny=None
rubert_tiny_full_finetuned=None
torch.cuda.empty_cache()
gc.collect()

33

### Задание 5 (1 балл)

Обучите *tbs17/MathBert* (с замороженным backbone и без заморозки), проанализируйте результаты. Сравните скоры с первым заданием. Получилось лучше или нет? Почему?

In [26]:
training_args_math = TrainingArguments(
    output_dir="math_model",
    learning_rate=2e-5,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none",
)
tokenizer_math = AutoTokenizer.from_pretrained("tbs17/MathBert")

math_model = TransformerClassificationModel.make("tbs17/MathBert",num_labels,id2label,label2id)
math_model_finetune = train_transformer(math_model,ds,tokenizer_math,compute_metrics,training_args_math,[])



tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/441M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tbs17/MathBert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,No log,1.186391,0.354385
2,No log,1.093022,0.371703
3,1.199000,1.061946,0.416542
4,1.199000,1.085384,0.442519
5,0.898500,1.104344,0.453577


In [52]:
training_args_math = None
math_model_finetune=None
math_model = None 
# tokenizer_math = None
torch.cuda.empty_cache()
gc.collect()

11901

### Задание 6 (1 балл)

Напишите функцию для отрисовки карт внимания первого слоя для моделей из задания

In [55]:
!ls math_model/checkpoint-844

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


config.json	   scheduler.pt		    trainer_state.json
model.safetensors  special_tokens_map.json  training_args.bin
optimizer.pt	   tokenizer.json	    vocab.txt
rng_state.pth	   tokenizer_config.json


In [75]:
mm = AutoModelForSequenceClassification.from_pretrained("math_model/checkpoint-844",num_labels=num_labels, id2label=id2label, label2id=label2id,output_attentions=True)

In [84]:
mm = mm.to(device)

In [36]:
!pip install bertviz botocore >/dev/null

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [90]:
from bertviz.neuron_view import show
from bertviz import head_view, model_view


model_type = 'bert'
"""
class BertConfig(object):

    def __init__(self,
                   vocab_size,
                   hidden_size=768,
                   num_hidden_layers=12, !!!!!!!!!!
                   num_attention_heads=12, !!!!!!!!!!
                   intermediate_size=3072,
                   hidden_act="gelu",
                   hidden_dropout_prob=0.1,
                   attention_probs_dropout_prob=0.1,
                   max_position_embeddings=512,
                   type_vocab_size=16,
                   initializer_range=0.02):
               
"""

def draw_first_layer_attention_maps(text: str, tokenizer, model):
    inputs = tokenizer.encode_plus(text,text,return_tensors='pt').to(device)
    input_ids = inputs['input_ids'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    attention = mm(input_ids, token_type_ids=token_type_ids)[-1]
    sentence_b_start = token_type_ids[0].tolist().index(1)
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list) 
    head_view(attention, tokens, sentence_b_start)


In [91]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


math_model  rubert_tiny  rubert_tiny_freeze


In [92]:
ds['test']['text'][0]

'Обозначим через S(x) сумму цифр натурального числа x. Решить уравнения: \n \xa0 а) \xa0x + S(x) + S(S(x)) = 1993;\n \xa0 б) \xa0x + S(x) + S(S(x)) + S(S(S(x))) = 1993. '

In [ ]:
draw_first_layer_attention_maps('Обозначим через S(x) сумму цифр натурального числа x. Решить уравнения: \n \xa0 а) \xa0x + S(x) + S(S(x)) = 1993;\n \xa0 б) \xa0x + S(x) + S(S(x)) + S(S(S(x))) = 1993. ',tokenizer_math,mm)

### Задание 7 (1 балл)

Проведите инференс для всех моделей **ДО ДООБУЧЕНИЯ** на 2-3 текстах из датасета. Посмотрите на головы Attention первого слоя в каждой модели на выбранных текстах (отрисуйте их отдельно).

Попробуйте их проинтерпретировать. Какие связи улавливают карты внимания? (если в модели много голов Attention, то проинтерпретируйте наиболее интересные)

In [31]:
### YOUR CODE IS HERE

### Задание 8 (1 балл)

Сделайте то же самое для дообученных моделей. Изменились ли карты внимания и связи, которые они улавливают? Почему?

In [32]:
### YOUR CODE IS HERE